## VNOŘENÝ SELECT

Vybere 1 jako SUBSELECT

In [ ]:
SELECT vnoreny.a 
FROM (SELECT 1 AS a) AS vnoreny;

Vybere jen některé sloupce jako SUBSELECT

In [ ]:
SELECT * 
FROM (
        SELECT gname, iyear 
        FROM czechitas.teror2 
        WHERE country = 54
      ) AS subselect;

Vybere unikátní dvojice skupiny a země jako SUBSELECT 

In [ ]:
SELECT vnoreny.* 
FROM (
        SELECT DISTINCT t.gname AS skupina, c.name AS zeme 
        FROM czechitas.teror2 AS t 
        INNER JOIN czechitas.country AS c ON t.country=c.id
     ) AS vnoreny;

Zobrazení všech teroristických událostí, které spáchala teroristická organizace s nejvetším počtem obětí

In [ ]:
SELECT gname, iyear, nkill 
FROM teror
WHERE gname = (SELECT TOP 1 gname FROM teror ORDER BY nkill DESC);

Počet mrtvých v letech 2017 a 2016, které má na svědomí Islámský Stát tak, aby ve výsledku byl název organizace a ve sloupcích počet mrtvých dle let

In [ ]:
SELECT t1.*, 
    t2.pocet_mrtvych_2016 
FROM (
        SELECT gname, SUM(nkill) AS pocet_mrtvych_2017
        FROM teror
        WHERE iyear=2017 AND gname like '%islamic state%' 
        GROUP BY gname
      ) AS t1
LEFT JOIN (
            SELECT gname, SUM(nkill) AS pocet_mrtvych_2016
            FROM teror
            WHERE iyear=2016
            GROUP BY gname
          ) AS t2
ON t1.gname = t2.gname;

Výběr teroristických útoků v roce 2016, které má na svědomí Islámský Stát a doplnění informace o max a min počtu obětí v roce 2016 ke každému útoku

In [ ]:
SELECT t1.eventid, t1.gname, t1.iyear, t2.max_mrtvych_2016, t2.min_mrtvych_2016
FROM teror AS t1
LEFT JOIN (
            SELECT gname, MAX(nkill) AS max_mrtvych_2016, min(nkill) AS min_mrtvych_2016
            FROM teror
            GROUP BY gname
          ) AS t2
ON t1.gname=t2.gname
WHERE t1.gname LIKE '%islamic state%' AND t1.iyear=2016;

## CTE

- Common Table Expressions

In [ ]:
/*
WITH prvni_nazev_poddotazu AS (
    SELECT  sloupec_x, 
            count(*) AS vse1 
    FROM tabulka1 
    GROUP BY sloupec_x
),
druhy_nazev_poddotazu AS (
    SELECT  sloupec_y, 
            count(*) AS vse2 
    FROM tabulka2 
    GROUP BY sloupec_y
)
SELECT  sloupec_x, 
        vse1 - vse2 AS rozdil
FROM prvni_nazev_poddotazu AS p
LEFT JOIN druhy_nazev_poddotazu AS d
ON p.sloupec_x = d.sloupec_y;
*/

Ukázka 1

In [ ]:
WITH ctepoddotaz AS (
    SELECT 1 AS jednicka, 'milion' AS dvojka
)
SELECT  c.jednicka,
        c.dvojka 
FROM ctepoddotaz c;

Ukázka 2

In [ ]:
WITH terorcountry AS (
    SELECT DISTINCT t.gname skupina,
                    c.name zeme 
    FROM czechitas.teror2 t 
    INNER JOIN czechitas.country c ON t.country=c.id
)
SELECT * FROM terorcountry;

Ukázka 3

In [ ]:
WITH cte AS (
    SELECT gname, iyear 
    FROM czechitas.teror2 
    WHERE country = 54
)
SELECT * FROM cte;

Ukázka 4

In [ ]:
WITH rukojmi_po_letech_fake AS (
     SELECT iyear, 
            SUM(nhostkid) AS rukojmi_fake 
     FROM teror 
     WHERE weaptype1_txt = 'Fake Weapons' AND nhostkid <> -99 
     GROUP BY iyear
), 
rukojmi_po_letech_bez_fake AS (
    SELECT  iyear, 
            SUM(nhostkid) AS rukojmi_bez_fake  
    FROM teror 
    WHERE weaptype1_txt <> 'Fake Weapons' AND nhostkid <> -99 
    GROUP BY iyear
)
 -- spojení přes roky
 SELECT f.iyear, 
        f.rukojmi_fake, 
        bf.rukojmi_bez_fake
 FROM rukojmi_po_letech_fake AS f 
 LEFT JOIN rukojmi_po_letech_bez_fake AS bf
 ON f.iyear = bf.iyear;